In [ ]:
from google.colab import drive
drive.mount('/content/drive') # 구글 드라이브에 있는 데이터 가져오기 위해 dirve.mount

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import random

In [ ]:
ttt_data = pd.read_csv('drive/MyDrive/data/tic-tac-toe.csv') # csv 파일 불러오기
print("데이터 변환 전 데이터 = ",ttt_data) # 변환 전 ttt_data 출력하기

# 문자인 데이터 값을 정수로 바꾼다
ttt_mapping = {'x': 2, 'o': 3, 'b':4, True :0, False: 1}
ttt_data['TL'] = ttt_data['TL'].map(ttt_mapping)
ttt_data['TM'] = ttt_data['TM'].map(ttt_mapping)
ttt_data['TR'] = ttt_data['TR'].map(ttt_mapping)
ttt_data['ML'] = ttt_data['ML'].map(ttt_mapping)
ttt_data['MM'] = ttt_data['MM'].map(ttt_mapping)
ttt_data['MR'] = ttt_data['MR'].map(ttt_mapping)
ttt_data['BL'] = ttt_data['BL'].map(ttt_mapping)
ttt_data['BM'] = ttt_data['BM'].map(ttt_mapping)
ttt_data['BR'] = ttt_data['BR'].map(ttt_mapping)
ttt_data['class'] = ttt_data['class'].map(ttt_mapping)

print("데이터 변환 후 데이터 = ", ttt_data)

In [ ]:
convertData = pd.DataFrame(ttt_data) # 변환한 데이터를 ttt_data를 convertData에 저장한다
convertData.to_csv('convert_data.csv') # convertData를 convert_data.csv에 저장한다

In [ ]:
def load_ttt(shuffle=True):   
    data = np.loadtxt("convert_data.csv", skiprows=1, delimiter=',') # convert_data.csv 읽어온다, 맨 위의 행 1 skip, 구분자 ','
    if shuffle == True:
        np.random.shuffle(data) # data를 난수로 섞음
    return data

convert_data = load_ttt() # convert_data에 데이터 불러온다

print("load_ttt() 후 데이터 = ", convert_data)

In [ ]:
def train_test_data_set(convert_data, test_rate=0.4): # train: 0.6, test: 0.4
    n = int(convert_data.shape[0]*(1-test_rate))
    x_train = convert_data[:n, 1:-1] 
    y_train = convert_data[:n, -1]
    
    x_test = convert_data[n:, 1:-1]
    y_test = convert_data[n:, -1]
    return (x_train, y_train), (x_test, y_test)

(x_train, y_train), (x_test, y_test) = train_test_data_set(convert_data, test_rate=0.4) # train_data 60%, test_data 40%
print("n:", int(convert_data.shape[0]*(1-0.4)))
print("x_train.shape:", x_train.shape) # 574x9
print("y_train.shape:", y_train.shape) # 574x1
print("x_test.shape:",  x_test.shape) # 384x9
print("y_test.shape:",  y_test.shape) # 384x1

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train) # 원핫 인코딩
y_test = tf.keras.utils.to_categorical(y_test)
#print("y_train = ", y_train)
#print("y_test = ", y_test)

#학습-1

In [ ]:
n = 10  # number of neurons in a hidden layer
# 모델 생성
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=n, input_dim=9, activation='sigmoid'))
model.add(tf.keras.layers.Dense(units=2, activation='softmax'))
model.summary()

def MSE(y, t):
    return tf.reduce_mean(tf.square(y - t)) # (y - t)**2

CCE = tf.keras.losses.CategoricalCrossentropy()

# 모델 어떻게 학습 시킬 것인지 설정
opt = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(optimizer=opt, loss= MSE, metrics=['accuracy'])

# 모델 학습
ret = model.fit(x_train, y_train, epochs=100, verbose=2) # batch_size=32
print("len(model.layers):", len(model.layers))  # 2
loss = ret.history['loss']
plt.plot(loss)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

In [ ]:
# 손실 값, 정확도 판별
train_loss, train_acc = model.evaluate(x_train, y_train, verbose=2)
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)
y_pred = model.predict(x_train)
y_label = np.argmax(y_pred, axis = 1)
C = tf.math.confusion_matrix(np.argmax(y_train, axis = 1), y_label)
print("confusion_matrix(C):", C)

#학습-2

In [ ]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=n, input_dim=9, activation='sigmoid'))
model.add(tf.keras.layers.Dense(units=2, activation='softmax'))
model.summary()

def MSE(y, t):
    return tf.reduce_mean(tf.square(y - t)) # (y - t)**2

CCE = tf.keras.losses.CategoricalCrossentropy()

opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
model.compile(optimizer=opt, loss=CCE, metrics=['accuracy'])

ret = model.fit(x_train, y_train, epochs=200, verbose=2) # batch_size=32
print("len(model.layers):", len(model.layers))  # 2
loss = ret.history['loss']
plt.plot(loss)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

In [ ]:

train_loss, train_acc = model.evaluate(x_train, y_train, verbose=2)
test_loss, test_acc = model.evaluate(x_test,  y_test, verbose=2)
y_pred = model.predict(x_train)
y_label = np.argmax(y_pred, axis = 1)
C = tf.math.confusion_matrix(np.argmax(y_train, axis = 1), y_label)
print("confusion_matrix(C):", C)